# Sentiment Analysis using LSTM

In this notebook, we’ll build an **LSTM-based model for sentiment analysis**. LSTMs are useful for text classification tasks like predicting whether a review is positive or negative.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

print("TensorFlow version:", tf.__version__)

## Load IMDB Dataset
We’ll use the **IMDB movie reviews dataset** (50,000 reviews labeled positive or negative).

In [ ]:
max_features = 10000  # only keep top 10k words
max_len = 200         # cut reviews after 200 words

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

print("Training samples:", len(X_train))
print("Test samples:", len(X_test))

## Build LSTM Model

In [ ]:
model = Sequential([
    Embedding(input_dim=max_features, output_dim=128, input_length=max_len),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

## Train Model

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

## Evaluate Model

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy*100:.2f}%")

## Predict Sentiment on New Review

In [ ]:
# Example: predict sentiment for a sample review
word_index = imdb.get_word_index()
reverse_word_index = {v:k for k,v in word_index.items()}

def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i-3, '?') for i in encoded_review])

sample_review = X_test[0]
decoded = decode_review(X_test[0])
print("Review:", decoded[:200], "...")
prediction = model.predict(sample_review.reshape(1, -1))
print("Predicted Sentiment:", "Positive" if prediction[0][0] > 0.5 else "Negative")